In [ ]:
!pip install import_ipynb
!pip install bayesian-optimization
import import_ipynb
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/Intro to DS - Capstone/Code/src')
from utilities import *

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=59ed1d66bc57a897ec797711cc1d6dae30ca0ccf8ad40dbd5c75df0d50e06b8d
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=1b5444310096176b1bffb26a211996c099dc91b23c0f677efc3f140d5e19fa78
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization
Mounted at /content/gdrive
importing Jupyter notebook from utilities.ipynb
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.3 MB 42.4 MB/s 
     |████████████████████████████████| 15.4 MB 25.5 MB/s 


# 4. Client clustering
In this notebook we will explore if there are significantly differentiated groups of clients with a descriptive purpose. To do so we will utilize KMeans clustering on demographic variables such as gender, education, marital status and age and analyze if there are significantly different billing patterns in each cluster. 

## 4.1 Read necessary data
First we will import the required packages and load in memory the taiwanese banking data which will be manipulated in the upcoming sections

In [ ]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

data = pd.read_excel(raw_path + 'default of credit card clients.xlsx', header = 1)
data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


## 4.2 Prepare data for clustering
Since some of the variables we want to cluster on come in categorical form and in different measures we will preprocess them with a PCA that captures 99% of the intrinsic variance in the data in order to weight them equally while looking for clusters within the data

In [ ]:
data_to_cluster = data.copy()

data_to_cluster['male'] = (data_to_cluster['SEX'] == 1).astype(float)
data_to_cluster['gradschool'] = (data_to_cluster['EDUCATION'] == 1).astype(float)
data_to_cluster['university'] = (data_to_cluster['EDUCATION'] == 2).astype(float)
data_to_cluster['married'] = (data_to_cluster['MARRIAGE'] == 1).astype(float)
data_to_cluster['card_balance'] = data_to_cluster['LIMIT_BAL']
data_to_cluster['bill_amt'] = data_to_cluster[['BILL_AMT1',	'BILL_AMT2', 'BILL_AMT3',	'BILL_AMT4',	'BILL_AMT5',	'BILL_AMT6']].mean(axis = 1)
data_to_cluster['pay_amt'] = data_to_cluster[['PAY_AMT1',	'PAY_AMT2',	'PAY_AMT3',	'PAY_AMT4',	'PAY_AMT5',	'PAY_AMT6' ]].mean(axis = 1)

stdScal = StandardScaler()
data_to_cluster = pd.DataFrame(stdScal.fit_transform(data_to_cluster[['male', 'university', 'married', 'card_balance', 'bill_amt', 'pay_amt']]))
data_to_cluster.head()

,0,1,2,3,4,5
0,-0.810161,1.0669,1.093780,-1.136720,-0.690692,-0.509027
1,-0.810161,1.0669,-0.914261,-0.365981,-0.665997,-0.438153
2,-0.810161,1.0669,-0.914261,-0.597202,-0.443170,-0.339216
3,-0.810161,1.0669,1.093780,-0.905498,-0.101507,-0.382454
4,1.234323,1.0669,1.093780,-0.905498,-0.422920,0.450421


## 4.3 Determine optimal number of clusters

In [ ]:
for clust in range(2, 40):
    km = KMeans(clust, random_state = 1234)
    silhouette_avg = silhouette_score(data_to_cluster, km.fit_predict(data_to_cluster))
    print("For n_clusters =",  clust, 
          "The average silhouette_score is :", silhouette_avg)

For n_clusters = 2 The average silhouette_score is : 0.21801850029671146
For n_clusters = 3 The average silhouette_score is : 0.23243869334737488
For n_clusters = 4 The average silhouette_score is : 0.2457638415446413
For n_clusters = 5 The average silhouette_score is : 0.29236725314614953
For n_clusters = 6 The average silhouette_score is : 0.3025960194467969
For n_clusters = 7 The average silhouette_score is : 0.3110816720111172
For n_clusters = 8 The average silhouette_score is : 0.34851705075447414
For n_clusters = 9 The average silhouette_score is : 0.3649800648426808
For n_clusters = 10 The average silhouette_score is : 0.3988546167777284
For n_clusters = 11 The average silhouette_score is : 0.4000302151577881
For n_clusters = 12 The average silhouette_score is : 0.42861652124444116
For n_clusters = 13 The average silhouette_score is : 0.4313153701241482
For n_clusters = 14 The average silhouette_score is : 0.4301326785495434
For n_clusters = 15 The average silhouette_score is : 

## 4.4 Analyze characteristics of each cluster
Having determined the optimal number of clusters we will build some comprehension around which are the individuals that fall in each one of them

In [ ]:
km = KMeans(29, random_state = 1234)
data['cluster'] = km.fit_predict(data_to_cluster)

summary_report = pd.DataFrame()

for clust in data.cluster.unique():
  
  subset = data[data.cluster == clust].reset_index(drop = True)

  current = pd.DataFrame({
      'Cluster number': [clust],
      'Number members': [subset.shape[0]],
      '% Males': [(subset.SEX == 1).mean()],
      '% gradschool': [(subset.EDUCATION == 1).mean()],
      '% university': [(subset.EDUCATION == 2).mean()],
      '% high school': [(subset.EDUCATION == 3).mean()],
      'Avg age': [(subset.AGE.mean())],
      'Avg card balance': [(subset.LIMIT_BAL.mean())],
      'Avg default rate': [(subset['default payment next month'].mean())]
      })
  
  summary_report = pd.concat([summary_report, current])

summary_report.sort_values('Number members', ascending = False, inplace = True)
summary_report.head(5)

,Cluster number,Number members,% Males,% gradschool,% university,% high school,Avg age,Avg card balance,Avg default rate
0,6,2865,0.0,0.710646,0.0,0.267016,30.591623,85123.909250,0.225480
0,2,2433,0.0,0.000000,1.0,0.000000,36.760789,78337.854501,0.279901
0,8,2418,0.0,0.000000,1.0,0.000000,29.504963,70616.211745,0.257237
0,1,2187,0.0,0.367627,0.0,0.600823,40.585734,101152.263374,0.245542
0,23,2152,1.0,0.627323,0.0,0.345260,32.247212,74196.096654,0.253717


In [ ]:
summary_report[['Number members', 'Avg default rate']]

,Number members,Avg default rate
0,2865,0.225480
0,2433,0.279901
0,2418,0.257237
0,2187,0.245542
0,2152,0.253717
0,2121,0.285714
0,1682,0.297860
0,1633,0.134721
0,1528,0.301702
0,1121,0.148974
